In [1]:
import os
from GPTModel.GPTModel import *
from GPTModel.ModelConfig import *
from GPTModel.Device import *
from ProteinStructureDataSet import *

In [2]:
#load the pretrained model
def load_model(model_path, cfg):
    model = GPTModel(cfg)
    model.load_state_dict(torch.load(model_path, weights_only=True))
    model.eval()
    return model

In [3]:
# load the pretrained model
CHOOSE_MODEL = "gpt2-small (124M)"
cfg, _ = GetModelConfig(CHOOSE_MODEL)
file_name = "TrainedModels/TrainedModel_49.pth"
model = load_model(file_name, cfg)

#load the sequences data set
data_file_name = os.path.join("Datasets", f"final_dataset.dat")
with open(data_file_name, "rb") as infile:
    uniref50_data = torch.load(infile)
uniref50_data = uniref50_data - 1

# split to train and test sets
sample_count = uniref50_data.shape[0]
train_set_ratio = 0.9
train_set_size = train_set_ratio * sample_count
train_data = uniref50_data[:int(train_set_size), :]
test_data = uniref50_data[int(train_set_size):, :]
# get a cuda device if the system supports it
device = GetDevice()
model.to(device);

In [5]:

correct_predictions_count = 0
test_samples_count = 1000
#iterate over samples from the test set
for i in range(test_samples_count):
    sample = torch.unsqueeze(test_data[i][:1024],dim = 0).long() # get the input sequence
    target = test_data[i][-1] # get the last token of the target data, which is the token that the model needs to predict
    sample_device = sample.to(device)
    target_device = target.to(device)
    logits = model(sample_device)
    prediction = torch.argmax(logits[0,-1,:], dim = -1) # find the token with the highest probability, which is the prediction of the model

    if prediction == target: # if the prediction is correct, add it to the count
        correct_predictions_count += 1

In [6]:
#print the prediction accuracy
print(correct_predictions_count/test_samples_count)

0.272
